<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2

### <center> Автор материала: Елена Девятайкина (@elenadevyataykina)

# JusticeHunter или как понять, преступление ли это или нелепая случайность

Я часто вижу статьи о том, что теперь есть возможность определить, добьется ли человек потенциальных успехов в профессиональной или творческой сфере, грозит ли ему болезнь Альцгеймера или Паркинсона лет так через 50, как найти вторую половину, чтобы сразу и на всю жизнь, а также вообще, что за человек и какой ориентации. 

Но что касается исследований психологических заболеваний, понимания мотивации человека к тому или иному действию, в частности к преступлениям, уже информации не так много (возможна она недоступна или же действительно здесь сделать выводы сложнее). Но так как в тюрьмы к заключенным отправляют огромное количество консультантов для бесед с ними, а также для выяснения более подробной информации о них, я склонна предполагать, что здесь слишком много надо информации изучить, прежде чем приступить к ее интерпретации. 

И вот, прочитав книгу **Mind Hunter: Inside FBI’s Elite Serial Crime Unit** и посмотрев сериал **MindHunter** (рекомендую для любителей криминалистики, ведь его снимал сам ФИНЧЕР), я решила сделать свое исследование на эту тему.

Изначально я рассчитывала получить данные из университета Рэдфорда (это, пожалуй, один из самых крупных датасетов, где собрано множество информации о серийных убийцах за 30 лет проведения данного исследования), но не срослось (если у кого-то есть эти данные, то я буду безумно благодарна, если вы поделитесь ими со мной).

Вместо этого был найден датасет на kaggle, где собрана информация о преступлениях, тех, кто их совершил и пострадавших (жертвах). Здесь рассматриваются данные из отделений полиции нескольких штатов Америки о преступлениях, совершенных с 1980 - 2014. Целью проекта является понять, можно ли отличить преднамеренное преступление от причинения вреда по неосторожности, а также понять, какие из предикторов являются наиболее значимыми при определении типа преступления и достаточно ли их вообще.

In [ ]:
# Отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'darkgrid') #зададим базовое оформление
%pylab inline

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.utils import shuffle

## Описание данных

Возьмем из данных только те столбцы, которые будут важны для дальнейшего исследования
(также в данных содержится информация о полиции, сроке расследования, а также о человеке, проводившем допрос)

In [ ]:
df = pd.read_csv('dataframe.csv', usecols=['Year', 'Month', 'Victim Sex', 'Victim Age', 'Victim Race', 'Perpetrator Sex', 'Perpetrator Age', 'Perpetrator Race', 'Relationship', 'Weapon', 'Victim Count', 'Crime Type'])

Посмотрим как выглядят данные

In [ ]:
df.head()

- Year - год совершения преступления
- Month - месяц совершения преступления
- Victim Sex - пол жертвы
- Victim Age - возраст жертвы
- Victim Race - раса жертвы
- Perpetrator Sex - пол исполнителя
- Perpetrator Age - возраст исполнителя
- Perpetrator Race - раса исполнителя
- Relationship - Родственные связи между исполнителем и жертвой
- Weapon - оружие
- Victim Count - количество жертв
- Crime Type - целевой признак, убийство (преднамеренное нанесение вреда) или приченение вреда по неосторожности

In [ ]:
df.info()

Как видно из данных есть несколько категориальных признаков, поэтому воспользуемся Label Encoding

In [ ]:
categorical_columns = df.columns[df.dtypes == 'object']
numerical_columns = ['Year', 'Victim Age', 'Perpetrator Age', 'Victim Count']

In [ ]:
label_encoder = LabelEncoder()
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])
df.head()

Посмотрим на корреляцию между признаками

In [ ]:
corr_features = df.drop('Crime Type', axis=1).corr()

In [ ]:
sns.set(font_scale=1.5)
#cmap = sns.diverging_palette(220, 10, as_cmap=True)
f, ax = plt.subplots(figsize=(16, 9))
sns.heatmap(corr_features, cmap='viridis', annot=True, ax=ax, fmt='.2f')

Видим, что достаточно высокая корреляция между расой жертвы и преступника

Посмотрим на соотношение типа совершаемых убийств между женщинами и мужчинами, где
- 0 - причинение вреда по неосторожности
- 1 - убийство или преднамеренное нанесение ущерба

In [ ]:
plt.figure(figsize(16,6))
plt.subplot(121)
stat_red = df[df['Perpetrator Sex'] == 1].groupby('Crime Type')['Crime Type'].agg(lambda x: float(len(x))/df[df['Perpetrator Sex'] == 1].shape[0])
stat_red.plot(kind='bar', color='g', width=0.9)
plt.xticks(rotation=0)
plt.ylabel('Type of crime', fontsize=14)
plt.xlabel('Men', fontsize=14)

plt.subplot(122)
stat_white = df[df['Perpetrator Sex'] == 0].groupby('Crime Type')['Crime Type'].agg(lambda x: float(len(x))/df[df['Perpetrator Sex'] == 2].shape[0])
stat_white.plot(color='y', kind='bar', width=0.9)
plt.xticks(rotation=0)
plt.ylabel('Type of crime', fontsize=14)
plt.xlabel('Women', fontsize=14)

В данных есть те преступления, где виновный неизвестен и их возраст указан как 0. Я уберу из выборки такие данные

In [ ]:
df = df[df['Perpetrator Age'] != 0]

Дополнительные данные можно получить только с помощью дополнительных сведений о преступнике или жертве и характере преступления, поэтому дальнейшие действия будут производиться на тех данных, которые есть

## Перейдем к обучению модели

### Random Forest

Сначала попробуем Random Forest, так как он не требует детальную предварительную настройку, а также есть возможность выявить степень предиктора, что нам и необходимо согласно поставленной цели

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

Так как отдельно тестовой выборки нет, разделим все данные на тестовые и валидационные в соотношении 70:30 и обучим **Random Forest** с параметрами `n_estimators=100, max_depth=15, max_features=5`

In [ ]:
sns.set(style = 'ticks', color_codes = True)

df = shuffle(df)
class_label = df.pop('Crime Type')
X_train, X_valid, y_train, y_valid = train_test_split(df, class_label, test_size = .7)

In [ ]:
%%time
model_rfc = RandomForestClassifier(n_estimators=100, max_depth=15, max_features=5, random_state=17, n_jobs=-1).\
                               fit(X_train, y_train)
print(classification_report(y_valid, model_rfc.predict(X_valid)))
plt.figure(figsize(16,8))
pd.Series(model_rfc.feature_importances_, index=X_valid.columns).plot(kind='barh', color='g')
plt.show()

Получается достаточно высокий уровень точности, но практически нет информации о значимости предикторов, так как все признаки находятся на одинаково небольшом уровне

### CatBoost

Попробуем другие алгоритмы, которые способны также показать степень важности предикторов. Следующим будет CatBoost. Используем его без преднасроенных гиперпараметров

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
model_cbc = CatBoostClassifier().fit(X_train, y_train)

print(classification_report(y_valid, model_cbc.predict(X_valid)))
pd.Series(model_cbc.get_feature_importance(X_valid, y_valid), index = X_valid.columns)

Из полученных результатов видно, что больший вес отдается признаку `Weapon`, хотя в `Random Forest` признак с наибольшим весом - `Victim Age`

### XGBoost

Для сравнения проделаем аналогичную операцию с **XGBoost**

In [ ]:
import xgboost as xgb

In [ ]:
model_xgb = xgb.XGBClassifier().fit(X_train, y_train)

print(classification_report(y_valid, model_xgb.predict(X_valid)))
importance = model_xgb.booster().get_score(importance_type = 'gain')
pd.Series(list(importance.values()), index = importance.keys())

xgb.plot_importance(model_xgb, color='green')
plt.show()

Здесь также видно, что самым значимым признаком является `Weapon`, после которого почти сразу же следует `Relationship`, что выглядит вполне логично



### KNeighborsClassifier

В задачах анализа поведенческих факторов часто используется алгоритм ближайших соседей. Это связано с тем, что расстояние (например, евклидова метрика) между двумя векторами похожих людей будет близким. Логично предположить, что люди одной группы набирают похожее количество баллов, возможно, формируя несколько кластеров. 

Данный алгоритм не позволяет показывать степень важности предикторов. Но в небольших наборах данных обычно вручную удаляются признаки, а ддалее смотрят на качество обучения модели без него.

Алгоритм ближайших соседей весьма чувствителен к разной природе данных (и к выбросам), следовательно, аналогично алгоритмам на основе нейронных сетей или методу опорных векторов требует предварительную нормализацию

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.fit_transform(X_valid)

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=7, n_jobs=-1).fit(X_train_scaled, y_train)
print(classification_report(y_valid, model_knn.predict(X_valid_scaled)))

### Tensorflow

Ради эксперимента посмотрим на результат алгоритма на основе нейронных сетей (указано 3000 шагов для ускорения расчетов)

In [ ]:
%%time
import tensorflow as tf

def get_train_inputs():
    xr = tf.constant(X_train_scaled)
    yr = tf.constant(y_train)
    return xr, yr

def get_test_inputs():
    xr = tf.constant(X_valid_scaled)
    yr = tf.constant(y_valid)
    return xr, yr

model_tf = tf.contrib.learn.DNNClassifier(feature_columns = [tf.contrib.layers.real_valued_column('', dimension = 4)],
                                          hidden_units = [10, 20, 10], n_classes = 2, model_dir = 'tmp_dir_tensorflow').\
                            fit(input_fn = get_train_inputs, steps=3000)

predictions = list(model_tf.predict(input_fn = get_test_inputs))
print(classification_report(y_valid, predictions))

## Выводы

Теперь можно перейти к выводам: 
    
в результате данного исследования было выявлено, что по представленному вектору можно с достаточно высокой степенью точности назвать правильный класс наблюдения, но все же недостаточной, если речь идет о том, чтобы быть посаженным на несколько лет. Поэтому в здачах подобного рода надо большое внимание уделить сбору данных, а также корректной интерпретации.

Были рассмотрены алгоритмы

- RandomForestClassifier
- CatBoostClassifier
- xgboost
- KNeighborsClassifier
- tensorflow

Что касается признаков, то удалось выделить те, которые имеют наибольший вес из имеющихся, но все же для такого исследования признаков недостаточно. 
Если бы была возможность, то я бы еще точно добавила информацию об образовании, о семье подозреваемого, а также ответы на специально подобранные вопросы в зависимости от характера преступления. 

---
**Если у вас есть вопросы, критика или вы хотите поделиться своими мыслями на этот счет, то в slack вы можете меня найти как @elenadevyataykina**